# Practice notebook for confidence intervals using NHANES data

This notebook will give you the opportunity to practice working with confidence intervals using the NHANES data.

You can enter your code into the cells that say "enter your code here", and you can type responses to the questions into the cells that say "Type Markdown and Latex".

Note that most of the code that you will need to write below is very similar to code that appears in the case study notebook.  You will need to edit code from that notebook in small ways to adapt it to the prompts below.

To get started, we will use the same module imports and read the data in the same way as we did in the case study:

In [47]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import re

da = pd.read_csv("nhanes_2015_2016.csv")

## Question 1

Restrict the sample to women between 35 and 50, then use the marital status variable [DMDMARTL](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDMARTL) to partition this sample into two groups - women who are currently married, and women who are not currently married.  Within each of these groups, calculate the proportion of women who have completed college.  Calculate 95% confidence intervals for each of these proportions.

In [108]:
# enter your code here

# making data human readable
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

# extrapoliting female
dfemale = da[da['RIAGENDRx'] == 'Female']

# extrapolating age range between 35 and 50
dfemale['agerange'] = pd.cut(dfemale['RIDAGEYR'], (35,50))
dfemale_age = dfemale[dfemale['agerange'].notna()]

# making data human readable again (have to go over this again to know why regex doesn't work)
dfemale_age['DMDMARTL'] = dfemale_age['DMDMARTL'].replace({1: 'Married', 2: 'Not_Married', 3: 'Not_Married', 4: 'Not_Married', 5: 'Not_Married', 6: 'Not_Married', 77: np.nan, 99: np.nan})
dfemale_age['DMDEDUC2'] = dfemale_age['DMDEDUC2'].replace({5: 'Graduated', 1: 'Not_Graduated', 2: 'Not_Graduated', 3: 'Not_Graduated', 4: 'Not_Graduated', 9: np.nan})

# dropping nan values
data_clean = dfemale_age[['DMDMARTL', 'DMDEDUC2']].dropna()

# creating crosstab
crosstab = pd.crosstab(data_clean.DMDEDUC2, data_clean.DMDMARTL)
print(crosstab)

DMDMARTL       Married  Not_Married
DMDEDUC2                           
Graduated          154           67
Not_Graduated      265          254


In [109]:
# calculating sample proportion of graduated and sample size
prop_total = data_clean.groupby(data_clean.DMDMARTL).agg({"DMDEDUC2": [lambda x: np.mean(x=="Graduated"), np.size]})
prop_total.columns = ["Proportion", "Total_n"]
print(prop_total)

             Proportion  Total_n
DMDMARTL                        
Married        0.367542      419
Not_Married    0.208723      321


In [110]:
p = prop_total.Proportion.Married # Married proportion
n = prop_total.Total_n.Married # Total number of married
se_married = np.sqrt(p * (1 - p) / n)
print(f'Married standard error: {se_married}')

p = prop_total.Proportion.Not_Married # Not married proportion
n = prop_total["Total_n"].Not_Married # Total number of not married
se_not_married = np.sqrt(p * (1 - p) / n)
print(f'Not married standard error: {se_not_married}')

Married standard error: 0.02355386941414772
Not married standard error: 0.022682802482912778


__Q1a.__ Identify which of the two confidence intervals is wider, and explain why this is the case. 

In [115]:
ci_married = sm.stats.proportion_confint(154, 154+265)
ci_not_married = sm.stats.proportion_confint(67, 67+254)
print(ci_married)
print(ci_not_married)
ci_married_size = ci_married[1] - ci_married[0]
ci_not_married_size = ci_not_married[1] - ci_not_married[0]
print(ci_married_size, ci_not_married_size)

# ci of married woman sample is larger, because the proportion is closer to 1/2

(0.3213770303614961, 0.41370650185807434)
(0.16426526549807702, 0.2531802173679666)
0.09232947149657822 0.08891495186988957


__Q1b.__ Write 1-2 sentences summarizing these findings for an audience that does not know what a confidence interval is (the goal here is to report the substance of what you learned about how marital status and educational attainment are related, not to teach a person what a confidence interval is).

## Question 2

Construct 95% confidence intervals for the proportion of smokers who are female, and for the proportion of smokers who are male.  Then construct a 95% confidence interval for the difference between these proportions.

In [ ]:
# enter your code here

__Q2a.__ Discuss why it may be relevant to report the proportions of smokers who are female and male, and contrast this to reporting the proportions of males and females who smoke.

__Q2b.__ How does the width of the confidence interval for the difference of the two proportions compare to the widths of the confidence intervals for each proportion separately?

## Question 3

Construct a 95% interval for height ([BMXHT](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.htm#BMXHT)) in centimeters.  Then convert height from centimeters to inches by dividing by 2.54, and construct a 95% confidence interval for height in inches.  Finally, convert the endpoints (the lower and upper confidence limits) of the confidence interval from inches to back to centimeters   

In [ ]:
# enter your code here

__Q3a.__ Describe how the confidence interval constructed in centimeters relates to the confidence interval constructed in inches.

## Question 4

Partition the sample based on 10-year age bands, i.e. the resulting groups will consist of people with ages from 18-28, 29-38, etc. Construct 95% confidence intervals for the difference between the mean BMI for females and for males within each age band.

In [ ]:
# enter your code here

__Q4a.__ How do the widths of these confidence intervals differ?  Provide an explanation for any substantial diferences in the confidence interval widths that you see.

## Question 5

Construct a 95% confidence interval for the first and second systolic blood pressure measures, and for the difference between the first and second systolic blood pressure measurements within a subject.

In [ ]:
# enter code here

__Q5a.__ Based on these confidence intervals, would you say that a difference of zero between the population mean values of the first and second systolic blood pressure measures is consistent with the data?


__Q5b.__ Discuss how the width of the confidence interval for the within-subject difference compares to the widths of the confidence intervals for the first and second measures.

## Question 6

Construct a 95% confidence interval for the mean difference between the average age of a smoker, and the average age of a non-smoker.

In [ ]:
# insert your code here

__Q6a.__ Use graphical and numerical techniques to compare the variation in the ages of smokers to the variation in the ages of non-smokers.  

In [1]:
# insert your code here

__Q6b.__ Does it appear that uncertainty about the mean age of smokers, or uncertainty about the mean age of non-smokers contributed more to the uncertainty for the mean difference that we are focusing on here?